In [1]:
# imports
from pathlib import Path
from time import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from utils import (
    read_single_problem_from_path_as_adjacency,
    check_validity_for_adjacency
)
from constants import BASE_INSTANCES_FILES, OTHER_INSTANCES_FILES, OTHER_INSTANCES_BEST_KNOWN, BASE_INSTANCES_BEST_KNOWN
from heuristics import descending_degree_glutonous_heuristic
from meta_heuristics import base_vns_meta_heuristic, enhansed_base_vns_meta_heuristic

# Constants
ROOT_DIR = Path.cwd().parent
# Instances pathes
INSTANCES_DIR = ROOT_DIR / "instances"
BASE_INSTANCES_DIR = INSTANCES_DIR / "project_instances"
OTHER_INSTANCES_DIR = INSTANCES_DIR / "other_instances"


In [2]:
# Testing vns base heuristic on a graph
_, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=BASE_INSTANCES_DIR / "brock200_2.col"
)
starting_clique=descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
clique , _,_,_= base_vns_meta_heuristic(starting_clique=starting_clique, graph=graph, degrees=degrees, max_time=5)
print("Is the clique valid ? ", check_validity_for_adjacency(graph, clique))
print("Is the clique different ?", all(clique== starting_clique))


Is the clique valid ?  True
Is the clique different ? False


In [2]:
# Testing on all graphs and displaying results

# Display setup
columns = ["instance", "max_k", "start", "end", "bound", "gap","start/end diff","time of best", "iteration_of_best", "total_iterations"]
display_dataframe = pd.DataFrame({column: [] for column in columns})

# Running
for instance in BASE_INSTANCES_FILES[:3]:
    number_of_nodes, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=BASE_INSTANCES_DIR / instance
)
    for k in range(2, 10):
        starting_clique=descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
        clique, time_best_found, iteration_of_best, number_of_iterations = base_vns_meta_heuristic(starting_clique=starting_clique, graph=graph, degrees=degrees, max_time=10, biggest_neighbourhood_size=k)
    
        # Adding to display
        new_row = [
            instance,
            k,
            np.sum(starting_clique),
            np.sum(clique),
            BASE_INSTANCES_BEST_KNOWN[instance],
            "{:.1%}".format(1 - np.sum(clique)/BASE_INSTANCES_BEST_KNOWN[instance]),
            "{:.1%}".format(np.sum(np.where(starting_clique == clique, 0, 1))/ number_of_nodes),
            time_best_found,
            iteration_of_best,
            number_of_iterations
        ]

        display_dataframe.loc[len(display_dataframe)] = new_row

display_dataframe.to_csv("vns_base_results_for_base_instances_2.csv", sep=";")


In [3]:
# Testing on all graphs and displaying results

# Display setup
columns = ["instance", "max_k", "start", "end", "bound", "gap","start/end diff","time of best", "iteration_of_best", "total_iterations"]
display_dataframe = pd.DataFrame({column: [] for column in columns})

# Running
for instance in OTHER_INSTANCES_FILES:
    number_of_nodes, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=OTHER_INSTANCES_DIR / instance
)
    for k in range(2, 20, 3):
        starting_clique=descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
        clique, time_best_found, iteration_of_best, number_of_iterations = base_vns_meta_heuristic(starting_clique=starting_clique, graph=graph, degrees=degrees, max_time=20, biggest_neighbourhood_size=k)
    
        # Adding to display
        new_row = [
            instance,
            k,
            np.sum(starting_clique),
            np.sum(clique),
            OTHER_INSTANCES_BEST_KNOWN[instance],
            "{:.1%}".format(1 - np.sum(clique)/OTHER_INSTANCES_BEST_KNOWN[instance]),
            "{:.1%}".format(np.sum(np.where(starting_clique == clique, 0, 1))/ number_of_nodes),
            time_best_found,
            iteration_of_best,
            number_of_iterations
        ]

        display_dataframe.loc[len(display_dataframe)] = new_row

display_dataframe.to_csv("vns_base_results_for_other_instances_2.csv", sep=";")


In [5]:
# Testing on single graph and displaying results

# Display setup
columns = ["instance", "max_k", "start", "end", "bound", "gap","start/end diff","time of best", "iteration_of_best", "total_iterations"]
display_dataframe = pd.DataFrame({column: [] for column in columns})

# Running
for instance in ["brock800_4.txt"]:
    number_of_nodes, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=OTHER_INSTANCES_DIR / instance
)
    starting_clique=descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
    for k in range(2,20):
        clique, time_best_found, iteration_of_best, number_of_iterations = base_vns_meta_heuristic(starting_clique=starting_clique, graph=graph, degrees=degrees, max_time=8, biggest_neighbourhood_size=k)

        # Adding to display
        new_row = [
            instance,
            k,
            np.sum(starting_clique),
            np.sum(clique),
            OTHER_INSTANCES_BEST_KNOWN[instance],
            "{:.1%}".format(1 - np.sum(clique)/OTHER_INSTANCES_BEST_KNOWN[instance]),
            "{:.1%}".format(np.sum(np.where(starting_clique == clique, 0, 1))/ number_of_nodes),
            time_best_found,
            iteration_of_best,
            number_of_iterations
        ]

        display_dataframe.loc[len(display_dataframe)] = new_row

display_dataframe.to_csv("brock800_4_tests.csv", sep=";")


In [3]:
# Comparing our two starting metas

# Display setup
columns = [
    "instance",
    "method",
    "start",
    "end",
    "bound",
    "gap",
    "start/end diff",
    "time of best",
    "iteration_of_best",
    "total_iterations",
]
display_dataframe = pd.DataFrame({column: [] for column in columns})
max_time = 60
biggest_neighbourhood_size = 5
# Running
for instance in OTHER_INSTANCES_FILES:
    number_of_nodes, _, graph, degrees = read_single_problem_from_path_as_adjacency(
        instance_path=OTHER_INSTANCES_DIR / instance
    )
    starting_clique = descending_degree_glutonous_heuristic(
        graph=graph, degrees=degrees
    )
    # Base heuristic
    (
        clique,
        time_best_found,
        iteration_of_best,
        number_of_iterations,
    ) = base_vns_meta_heuristic(
        starting_clique=starting_clique,
        graph=graph,
        degrees=degrees,
        max_time=max_time,
        biggest_neighbourhood_size=biggest_neighbourhood_size,
    )
    # Adding to display
    new_row = [
        instance,
        "base",
        np.sum(starting_clique),
        np.sum(clique),
        OTHER_INSTANCES_BEST_KNOWN[instance],
        "{:.1%}".format(1 - np.sum(clique) / OTHER_INSTANCES_BEST_KNOWN[instance]),
        "{:.1%}".format(
            np.sum(np.where(starting_clique == clique, 0, 1)) / number_of_nodes
        ),
        time_best_found,
        iteration_of_best,
        number_of_iterations,
    ]
    display_dataframe.loc[len(display_dataframe)] = new_row

    # Heuristic with restarts
    (
        clique,
        time_best_found,
        iteration_of_best,
        number_of_iterations,
    ) = enhansed_base_vns_meta_heuristic(
        starting_clique=starting_clique,
        graph=graph,
        degrees=degrees,
        max_time=max_time,
        max_iterations_without_improvement=5,
        penalization=3,
        biggest_neighbourhood_size=biggest_neighbourhood_size,
    )
    # Adding to display
    new_row = [
        instance,
        "restart",
        np.sum(starting_clique),
        np.sum(clique),
        OTHER_INSTANCES_BEST_KNOWN[instance],
        "{:.1%}".format(1 - np.sum(clique) / OTHER_INSTANCES_BEST_KNOWN[instance]),
        "{:.1%}".format(
            np.sum(np.where(starting_clique == clique, 0, 1)) / number_of_nodes
        ),
        time_best_found,
        iteration_of_best,
        number_of_iterations,
    ]
    display_dataframe.loc[len(display_dataframe)] = new_row

display_dataframe.to_csv("vns_versus_enhanced_penalization_3.csv", sep=";")
